In [1]:
import re
import json

import pandas as pd
from notion.block import CollectionViewBlock

import helpers

In [58]:
from imp import reload
reload(helpers)

<module 'helpers' from '/Users/jrw/epidemics/containment/helpers.py'>

In [2]:
MEASURES_DB_URL = 'https://www.notion.so/977d5e5be0434bf996704ec361ad621d?v=fe54f89ca9e04ac799af42b39e1efc4b'
COMPLETENESS_DB_URL = 'https://www.notion.so/dbf0f4ffc03f4417a16d65a6d08eea69?v=845ef4baafd44e72b9c483381b90caae'

In [3]:
client = helpers.get_notion_client()

In [87]:
def get_tags():
    with open('data/tags.json', 'r') as fp:
        api_response = json.load(fp)
        return pd.DataFrame(api_response['data']['containment_tags'].values())

In [88]:
tags = get_tags()

In [4]:
measures_db = client.get_block(MEASURES_DB_URL)
completeness_db = client.get_block(COMPLETENESS_DB_URL)

In [ ]:
# Notion API seems to take a long time with large tables
measures = helpers.df_from_notion_table(measures_db)

In [59]:
completeness = helpers.df_from_notion_table(
    completeness_db)[['country', 'complete_up_to_date']].dropna()

In [93]:
md = measures.dropna()

In [68]:
cm = measures.merge(completeness, on='country')

In [106]:
cm = cm[cm.keywords.apply(len) > 0]

In [108]:
cm.keywords.apply(len).sum()

1967

In [109]:
len(cm)

1288

In [101]:
cm.loc[262]

target_region                                                                        []
source                                https://www.hongkongfp.com/2020/03/21/breaking...
keywords                              [public mask and hygiene supply, social distan...
country                                                                       Hong Kong
date_end_intended                                                   2020-04-24 00:00:00
implementing_state_province                                                          []
target_state                                                                           
implementing_city_municipality                                                       []
target_city                                                                            
applies_to                                                                             
date_start                                                          2020-03-21 00:00:00
exceptions                      

In [145]:
cm.implementing_city_municipality.apply(lambda x: None if len(x) == 0 else x).dropna()

5             [Ischgl]
89                  []
105            [Daegu]
106            [gunpo]
108            [Daegu]
             ...      
1249                []
1255                []
1296          [Bansko]
1302           [Sofia]
1321    [Vatican City]
Name: implementing_city_municipality, Length: 105, dtype: object

In [149]:
fields_to_use, renamed_fields = (list(cols) for cols in zip(
#     ('id', 'id'), # not reliable
    ('country', 'country'),
#     # maybe worth adding these:
#     ('implementing_state_province', 'state_or_province'),
#     ('implementing_city_municipality', 'municipality'),
#     ('target_country', 'target_country'),
    ('description_of_measure_implemented', 'description'),
    ('source', 'source'),
    ('date_start', 'start_date'),
    ('date_end_intended', 'end_date'),
    ('quantity', 'quantity'),
    ('keywords', 'tags')
))

# other columns were not include b/c too few rows have
# data to bother programming them into the frontend

In [150]:
mc2 = cm[fields_to_use].copy()
mc2.columns = renamed_fields

In [151]:
mc2

,country,description,source,start_date,end_date,quantity,tags
0,Austria,On 10 March 2020 government announced that all...,https://en.wikipedia.org/wiki/2020_coronavirus...,2020-03-16,NaT,NaN,[university closure]
1,Austria,All outdoor events with more than 500 people a...,https://en.wikipedia.org/wiki/2020_coronavirus...,2020-03-10,NaT,500.0,"[gathering of more than 100 people banned, out..."
2,Austria,"Tests carried out to date: 15,613",https://ourworldindata.org/coronavirus-testing...,2020-03-20,NaT,15613.0,[testing numbers total]
3,Austria,"As of 16 March, nationwide, homes may only be ...",https://en.wikipedia.org/wiki/2020_coronavirus...,2020-03-16,NaT,NaN,[household quarantine]
4,Austria,"Beginning from 1 March, authorities in Germany...",https://en.wikipedia.org/wiki/2020_coronavirus...,2020-03-01,NaT,NaN,[source tracing]
...,...,...,...,...,...,...,...
1320,Moldova,"All schools closed, including pre-schools",https://www.reuters.com/article/us-health-coro...,2020-03-11,NaT,NaN,"[school closure, university closure, nursery s..."
1321,Vatican City,Italy lockdown measures are also applied to th...,https://www.theguardian.com/world/video/2020/m...,2020-03-10,NaT,NaN,"[social distancing, religious activity limitat..."
1322,Macedonia,"On 26 February, North Macedonia confirmed its ...",https://en.wikipedia.org/wiki/2020_coronavirus...,2020-02-26,NaT,NaN,"[case reporting, first case]"
1323,Macedonia,temporary 2 week closure of all education inst...,https://en.wikipedia.org/wiki/2020_coronavirus...,2020-03-10,2020-03-24,NaN,"[school closure, university closure]"


In [152]:
def expand_tags(measures):
    tagged_rows = []
    for _, measure in measures.iterrows():
        for tag in measure['tags']:
            row = dict(tag=tag, **measure.to_dict())
            del row['tags']
            tagged_rows.append(row)
    return pd.DataFrame(tagged_rows)

In [154]:
tagged = expand_tags(mc2)

In [8]:
def default_values(kw):
    for k, v in {**MAX_COLS,**TEST_COLS}.items():
        if (kw in v) and (v[kw]!=np.nan):
            return v[kw]
    return np.nan

def keywords(kws_quants):
    res =  pd.DataFrame([(i,j[1]) 
                         for j in kws_quants.values 
                         for i in str(j[0]).split(', ')],
                        columns=['Keywords','Quantity'])
    res['Quantity'] = res['Keywords'].apply(default_values).fillna(res['Quantity'])
    return res

def sum_kws(kws_quants,tags):
    return pd.Series(kws_quants['Keywords'].unique()).isin(tags).sum()

def max_kws(kws_quants, tags):
    return kws_quants[kws_quants['Keywords'].isin(tags)]['Quantity'].max()

def min_kws(kws_quants,tags):
    return kws_quants[kws_quants['Keywords'].isin(tags)]['Quantity'].min()

def test_kws(kws_quants,tags):
    if 'test all' in kws_quants['Keywords']:
        return 1
    elif 'test symptomatic' in kws_quants['Keywords']:
        return 0.5
    else:
        return kws_quants[kws_quants['Keywords'].isin(tags)]['Quantity'].sum()


In [9]:
index = {}

for cat, tags in MAX_COLS.items():
    for tag, value in tags.items():
        index[tag] = {"type": "min", "cat": cat, "value": value}

for cat, tags in MIN_COLS.items():
    for tag, value in tags:
        index[tag] = {"type": "min", "cat": cat, "value": value}

for cat, tags in CUMSUM_COLS.items():
    for tag in tags:
        index[tag] = {"type": "cum", "cat": cat, "value": value}

ValueError: too many values to unpack (expected 2)

In [7]:
def do_country(df):
    exp = pd.DataFrame([(id,keyword) 
        for id, item in df.iterrows()
        for keyword in str(item["Keywords"]).split(', ')],
        columns=['ID', 'Keyword']).set_index("ID")

    df = df.join(exp).set_index("Keyword")

    out = {}

    for cat, tags in MAX_COLS.items():
        thing = df.loc[df.index.isin(tags.keys())]
        out[cat] = thing

    return out

g = cm.sort_values("Date Start").groupby("Country")
df = g.get_group("Italy");
print(df.columns)
for key, thing in do_country(df).items():
    print(key)
    print(thing[["Date Start", "Description of measure implemented"]].to_records())
    print()

Index(['ID', 'Applies To', 'Country', 'Date Start', 'Date end intended',
       'Description of measure implemented', 'Exceptions', 'Implementing City',
       'Implementing State/Province', 'Keywords', 'Quantity', 'Source',
       'Target city', 'Target country', 'Target region', 'Target state'],
      dtype='object')
isolation
[]

quarantine
[('blanket curfew - no symptoms', '2020-03-08T00:00:00.000000000', 'avoiding moves from place of residence')
 ('cluster isolation - no symptoms', '2020-02-22T00:00:00.000000000', 'On 22 February, the government announced a new decree imposing the quarantine of more than 50,000 people from 11 different municipalities in Northern Italy. The quarantine zones are called the Red Zones and the areas in Lombardy and Veneto outside of them are called the Yellow Zones.[150] Penalties for violations range from a €206 fine to three months of imprisonment.[151] The Italian military and law enforcement agencies were instructed to secure and implement the lock

In [68]:
data_dict = {}

g = cm.sort_values("Date Start").groupby("Country")
for name, df in g:
    do_country()

,ID,Applies To,Country,Date Start,Date end intended,Description of measure implemented,Exceptions,Implementing City,Implementing State/Province,Keywords,Quantity,Source,Target city,Target country,Target region,Target state,Keyword
217,76,NaN,Italy,2020-03-12,2020-03-25,"closure of shops (supermarkets, grocery, pharm...","supermarkets, grocery, pharmacy",NaN,NaN,general nonessential business suspension,NaN,https://www.mzv.cz/rome/cz/viza_a_konzularni_i...,NaN,NaN,NaN,NaN,general nonessential business suspension
218,77,NaN,Italy,2020-03-12,2020-03-25,closures of companies,"banks, transportations, financial and insuranc...",NaN,NaN,general nonessential business suspension,NaN,https://www.mzv.cz/rome/cz/viza_a_konzularni_i...,NaN,NaN,NaN,NaN,general nonessential business suspension
219,78,NaN,Italy,2020-03-08,NaT,avoiding moves from place of residence,NaN,NaN,NaN,blanket curfew - no symptoms,NaN,https://www.mzv.cz/rome/cz/viza_a_konzularni_i...,NaN,NaN,NaN,NaN,blanket curfew - no symptoms
220,80,NaN,Italy,2020-03-08,2020-04-04,"stop sport activities (gyms, swimming pools, t...",NaN,NaN,NaN,sports cancellation,NaN,https://www.mzv.cz/rome/cz/viza_a_konzularni_i...,NaN,NaN,NaN,NaN,sports cancellation
221,79,NaN,Italy,2020-03-08,2020-04-04,"stop cultural activities (cinemas, theaters, m...",NaN,NaN,NaN,cultural activity limitation,NaN,https://www.mzv.cz/rome/cz/viza_a_konzularni_i...,NaN,NaN,NaN,NaN,cultural activity limitation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,748,NaN,Italy,2020-03-07,NaT,"Shopping centres had to be closed on weekends,...",NaN,NaN,NaN,limited nonessential business suspension,NaN,https://en.wikipedia.org/wiki/2020_coronavirus...,NaN,NaN,NaN,NaN,limited nonessential business suspension
784,751,NaN,Italy,2020-03-08,NaT,Riots broke out in many penitentiaries through...,NaN,NaN,NaN,"prisons closed to visitors, riots",NaN,https://en.wikipedia.org/wiki/2020_coronavirus...,NaN,NaN,NaN,NaN,prisons closed to visitors
784,751,NaN,Italy,2020-03-08,NaT,Riots broke out in many penitentiaries through...,NaN,NaN,NaN,"prisons closed to visitors, riots",NaN,https://en.wikipedia.org/wiki/2020_coronavirus...,NaN,NaN,NaN,NaN,riots
788,747,NaN,Italy,2020-03-07,NaT,The decree also established the closure of all...,NaN,NaN,NaN,sports cancellation,NaN,https://en.wikipedia.org/wiki/2020_coronavirus...,NaN,NaN,NaN,NaN,sports cancellation
